In [37]:
import requests, json, numpy, datetime
import pywren
import boto3
import numpy as np
import botometer
import pandas as pd
import botometer_creds as creds


In [39]:
'''
Creates the botometer object used to inspect twitter accounts. 
'''
bot = botometer.Botometer(wait_on_ratelimit=True,
                          rapidapi_key=creds.RAPIDAPI_KEY,
                          **creds.TWITTER_APP_AUTH)

In [40]:
accounts = ['@clayadavis', '@onurvarol', '@dscovr_epic', '@drvox', '@Nextgov']

In [41]:
%%time

pwex = pywren.default_executor()
futures = pwex.map(bot.check_account, accounts)
all_bot_scores = pywren.get_all_results(futures)

CPU times: user 1.25 s, sys: 184 ms, total: 1.43 s
Wall time: 18.1 s


In [42]:
all_bot_scores

[{'cap': {'english': 0.004268306222830912, 'universal': 0.005778300973814037},
  'categories': {'content': 0.17800952303063913,
   'friend': 0.10612616098731911,
   'network': 0.07473755397913713,
   'sentiment': 0.21601031401999074,
   'temporal': 0.34186854357920926,
   'user': 0.05042452964186063},
  'display_scores': {'content': 0.9,
   'english': 0.5,
   'friend': 0.5,
   'network': 0.4,
   'sentiment': 1.1,
   'temporal': 1.7,
   'universal': 0.6,
   'user': 0.3},
  'scores': {'english': 0.10827700665723966, 'universal': 0.11956986651967942},
  'user': {'id_str': '1548959833', 'screen_name': 'clayadavis'}},
 {'cap': {'english': 0.004775575529426348, 'universal': 0.021041094138717433},
  'categories': {'content': 0.4839136138086953,
   'friend': 0.16863713142666406,
   'network': 0.15895834190828398,
   'sentiment': 0.20494668795721943,
   'temporal': 0.15332301110829857,
   'user': 0.08970732338544227},
  'display_scores': {'content': 2.4,
   'english': 0.6,
   'friend': 0.8,
   

In [25]:
def classify_account(dictionary):
    '''
    Given a dictionary that represents the botometer output, this function creates a binary
        variable indicating wheter or not the account is a bot based on a pre-determined
        threshold. 
    '''
    
    user_info = dictionary['user']
    name = user_info['screen_name']
    score = dictionary['scores']['english']
    bot = 0
    if score >= .29:
        bot += 1
    
    return {'account_name': name, 'bot': bot}
    
    

In [31]:
%%time

pwex = pywren.default_executor()
futures = pwex.map(classify_account, all_bot_scores)
bot_or_not = pywren.get_all_results(futures)

CPU times: user 1.09 s, sys: 150 ms, total: 1.24 s
Wall time: 21.1 s


In [32]:
bot_or_not

[{'account_name': 'clayadavis', 'bot': 0},
 {'account_name': 'onurvarol', 'bot': 0},
 {'account_name': 'dscovr_epic', 'bot': 0},
 {'account_name': 'drvox', 'bot': 0},
 {'account_name': 'Nextgov', 'bot': 1}]

In [34]:
pd.DataFrame(bot_or_not)

,account_name,bot
0,clayadavis,0
1,onurvarol,0
2,dscovr_epic,0
3,drvox,0
4,Nextgov,1
